## Useful links:

https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection

https://robjhyndman.com/hyndsight/crossvalidation/

https://towardsdatascience.com/time-series-nested-cross-validation-76adba623eb9


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_selection import SelectFromModel

In [59]:
dataset = pd.read_csv('../Data/imputedWQ.csv', parse_dates=True, header=0, index_col=0)

In [60]:
testset = pd.read_csv('../Data/testing2018-mean_imputed.csv', parse_dates=True, header=0, index_col=0)

In [61]:
cols = list(dataset.columns.values)
cols = cols[:-1]
dataset[cols] = dataset[cols].rolling(window=60).mean()
dataset = dataset.iloc[60:]

In [62]:
X = dataset.drop('EVENT', axis=1)  
y = dataset['EVENT']  

# TimeSeriesSplit
## Special case of KFold
### Param: n_splits according to the number of months

P.s: even though this probably would not result in accurate month separation, probably would go more with accurate samples separation

In [7]:
tss = TimeSeriesSplit(n_splits=4)
# kn = KNeighborsClassifier(n_neighbors=5, algorithm='brute') 
# sc = cross_val_score(kn, X, y, cv=tss, scoring='accuracy')

In [7]:
dataset.groupby(dataset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
8,290697.052632,6799.713362,343434.304450,3.095388e+07,8.662770e+06,786979.749453,4.008524e+06,6.737174e+07,3.885911e+07,165.0
9,353955.600000,7172.806251,361367.922324,3.252037e+07,9.111778e+06,865139.886451,4.396339e+06,6.858679e+07,4.056066e+07,1310.0
10,431429.980435,7384.744348,374076.598978,3.364092e+07,9.410066e+06,710265.240783,4.398468e+06,6.299556e+07,4.047151e+07,174.0
11,112343.550000,1736.255000,88672.950000,7.956982e+06,2.056477e+06,144182.960000,9.913322e+05,1.533778e+07,9.644876e+06,77.0


In [8]:
testset.groupby(testset.index.month).count()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
1,44640,44640,44640,44640,44640,44640,44640,44640,44640,44640
2,17641,17641,17641,17641,17641,17641,17641,17641,17641,17641
11,21293,21293,21293,21293,21293,21293,21293,21293,21293,32645
12,31512,31512,31512,31512,31512,31512,31512,31512,31512,44640


## TimeSeries CV

- Applying TimeSeriesSplit in cross validation with multiple scores
    - F1-macro (alter. f1_weighted, f1_micro)
    - Precision
    - Recall
    
P.s errors result because the classifier on specific folds couldn't identify at all one class and probably gave just true or false ( I guess false in this case because of high class-imbalance)

In [8]:
scoring = ['f1', 'f1_macro', 'f1_micro', 'precision', 'recall']

In [22]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=4, random_state=0)

results = cross_validate(rf_clf, X, y, cv=tss, scoring=scoring, n_jobs=-1)

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
print("Model(RF, 2): ({0}, {1}, {2}, {3},{4})\n".format(results['test_f1'].mean(),results['test_f1_micro'].mean(), results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(RF, 2): (0.4324767210713538, 0.9879348643903838, 0.7131665873748906, 0.5094543072962713,0.4474395153523134)



In [24]:
y.unique()

array([False,  True])

In [25]:
rf_clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
y_pred = rf_clf.predict(X)

In [27]:
f1_score(y_pred, y)

0.5784989858012171

## GridSearchCV
### Hyperparameter optimizations for RandForest classifier

- Scoring F1_macro can be changed to precision, recall or f1_weighted or f1_micro and possibly you would get different parameters.

In [28]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': np.arange(2, 200, 10),
              'max_depth': np.arange(2,10)}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=tss, scoring="f1", n_jobs=-1)

In [29]:
grid.fit(X,y)

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': array([  2,  12,  22,  32,  42,  52,  62,  72,  82,  92, 102, 112, 122,
       132, 142, 152, 162, 172, 182, 192]), 'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [30]:
grid.best_params_

{'max_depth': 4, 'n_estimators': 32}

In [31]:
model = grid.best_estimator_

In [63]:
results = cross_validate(model, X, y, cv=tss, scoring=scoring, n_jobs=-1)

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [64]:
print("Model(RF, 4, 32): ({0}, {1}, {2}, {3}, {4})\n".format(results['test_f1'].mean(),results['test_f1_micro'].mean() , results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(RF, 4, 32): (0.30780053816417796, 0.987437726246371, 0.6507067551666663, 0.40695646945646946, 0.28214854211935925)



## Testing on Test set

In [65]:
cols = list(testset.columns.values)
cols = cols[:-1]
testset[cols] = testset[cols].rolling(window=60).mean()
testset = testset.iloc[60:]

In [66]:
X_test = testset.drop('EVENT', axis=1)  
y_test = testset['EVENT'] 

In [67]:
predicted = model.predict(X_test)

In [68]:
predicted 

array([False, False, False, ..., False, False, False])

In [69]:
f1_score(y_test,predicted)

0.48271446862996165

In [70]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test ,predicted)

array([[115536,     41],
       [  1575,    754]])

In [71]:
print(f1_score(y_test,predicted))
print(recall_score(y_test,predicted))
print(precision_score(y_test,predicted))

0.48271446862996165
0.32374409617861744
0.9484276729559749
